# Database monthly stats

This notebook contains a basic set of methods for querying monthly article quality statistics. 

## Queries
These two queries do the basic database extraction.  `all_wiki_aggregation` query is a simple

In [2]:
all_wiki_aggregation_query = """
SELECT
  timestamp AS month,
  SUM(weighted_sum) AS weighted_sum,
  SUM(LOG(weighted_sum)) AS weighted_log_sum,
  SUM(prediction = "Stub") AS stub_n,
  SUM(prediction = "Start") AS start_n,
  SUM(prediction = "C") AS c_n,
  SUM(prediction = "B") AS b_n,
  SUM(prediction = "GA") AS ga_n,
  SUM(prediction = "FA") AS fa_n,
  COUNT(*) AS n
FROM datasets_p.monthly_wp10_enwiki
GROUP BY month;
"""

wikiproject_aggregation_query = """
SELECT
  monthly_aq.timestamp AS month,
  SUM(weighted_sum) AS weighted_sum,
  SUM(LOG(weighted_sum)) AS weighted_log_sum,
  SUM(prediction = "Stub") AS stub_n,
  SUM(prediction = "Start") AS start_n,
  SUM(prediction = "C") AS c_n,
  SUM(prediction = "B") AS b_n,
  SUM(prediction = "GA") AS ga_n,
  SUM(prediction = "FA") AS fa_n,
  COUNT(*) AS n
FROM enwiki.page AS talk
INNER JOIN enwiki.page AS article ON
  talk.page_title = article.page_title AND
  article.page_namespace = 0
INNER JOIN enwiki.templatelinks USE INDEX (tl_namespace) ON
  tl_from = talk.page_id
INNER JOIN datasets_p.monthly_wp10_enwiki AS monthly_aq ON
  article.page_id = monthly_aq.page_id
WHERE
  talk.page_namespace = 1 AND
  tl_namespace = 10 AND
  (
    tl_title = %(project_template)s OR 
    tl_title IN (
      SELECT page.page_title 
      FROM enwiki.pagelinks 
      INNER JOIN enwiki.page ON page_id = pl_from 
      WHERE 
        pl_namespace = 10 AND 
        pl_title = %(project_template)s AND 
        pl_from_namespace = 10 AND 
        page_is_redirect
    )
  )
GROUP BY month;
"""

## Database connection incantation

In [7]:
import pymysql 

conn = pymysql.connect(
    host="labsdb-analytics.eqiad.wmnet", read_default_file="~/.my.cnf", db="datasets_p")

## Define functions for gathering and writing aggregated data

In [8]:
def all_wiki_aggregation():
    """
    Generate a cross-wiki monthly-aggregate dataset.  
    Returns a cursor that iterates over tuples (rows of the result set).
    """
    with conn.cursor() as cursor:
        cursor.execute(all_wiki_aggregation_query)
        return cursor

In [9]:
def wikiproject_aggregation(project_template):
    """
    Genrerate a wikiproject-specific monthly-aggregate dataset.
    Returns a cursor that iterates over tuples (rows of the result set).
    """
    with conn.cursor() as cursor:
        cursor.execute(wikiproject_aggregation_query, 
                       {'project_template': project_template})
        return cursor

In [10]:
import csv 
def dump_aggregation(cursor, file):
    headers = [i[0] for i in cursor.description]
    writer = csv.writer(file, delimiter='\t', quoting=csv.QUOTE_NONE, fieldnames=headers)
    writer.writeheader()
    for row in cursor:
        writer.writerow(row)